In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from numpy.random import multivariate_normal as mvnorm
from numpy.linalg import norm

import matplotlib.pyplot as plt
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
%matplotlib inline

import pdb

seed = 0
np.random.seed(seed)

### Import and generate data

In [2]:
# Import data
data = pd.read_pickle("~/Downloads/integrated_draws_20_5_4.pkl")
n_draws, n_time, dim_X, dim_fac = data['beta'].shape
beta = data['beta']
X_sd = data['x_sd']

# Generate X
m_fac = np.zeros((dim_fac))
s_fac = np.diag(np.ones((dim_fac)))
fac = np.expand_dims(mvnorm(m_fac, s_fac, (n_draws, n_time)), -1)

X = np.matmul(beta, fac).squeeze()

m_eps = np.zeros((dim_X))
for i in range(n_draws):
    eps = mvnorm(m_eps, np.diag(X_sd[i] ** 2), n_time)
    X[i] += X[i] + eps
    
X = X.transpose((2, 0, 1))
X_cov = np.einsum('ijkl,jmkl->imkl', np.expand_dims(X, 1), np.expand_dims(X, 0))

### Train

In [44]:
from regain.covariance import EqualityTimeGraphicalLasso
tgl = EqualityTimeGraphicalLasso(max_iter=100, loss='DT', c_level=0.25, rho=1e3, psi="laplacian")
emp_inv_score, baseline_score, fit_score, pre = tgl.fit_cov(X_cov).eval_cov_pre()    

[-14.25935396 -14.1553728  -13.61161447 -12.43170196 -13.03759254
 -13.78971749 -14.09494857 -13.67734398 -14.2256799  -13.75787454
 -13.50976898 -13.81418987 -13.70243031 -15.12532809 -14.93561939
 -14.31598789 -13.75765699 -14.20661868 -13.38483872 -13.74800111
 -13.30904642 -13.12803684 -12.582789   -11.11580319 -12.56024183
 -12.17486003 -13.16005662 -12.49824547 -13.03279464 -13.26649493
 -13.33298411 -12.82368694 -12.92469208 -12.82274771 -13.56824831
 -12.34162239 -12.88226284 -13.4761087  -12.76059698 -13.05692413
 -13.12428488 -14.02107143 -13.22311291 -12.53891478 -13.07876931
 -12.98231034 -13.37039083 -12.8419272  -12.66723014 -13.04746756
 -12.69555837 -12.86376416 -13.08486734 -13.73203442 -14.2144382
 -12.73640555 -13.29991844 -12.97856122 -13.55820472 -14.65563733
 -14.12103796 -14.49582311 -13.75737482 -14.00907139 -13.2207042
 -14.06072204 -13.30172436 -14.10596209 -13.0567297  -13.79257913
 -12.88445876 -13.08899257 -13.68091636 -13.65659687 -13.30835231
 -14.2053301

[1.27452052e+04 1.31226375e+05 1.26407085e+05 4.81196608e+04
 1.50057982e+05 1.79248007e+05 1.52612874e+05 1.73783211e+05
 1.53320487e+05 1.55085196e+05 1.85367030e+05 1.36981363e+05
 1.01364773e+05 6.34149811e+04 6.46914039e+04 2.53360570e+05
 1.60409966e+05 4.76935958e+05 2.06789249e+05 1.04896073e+05
 1.48675635e+05 1.79269148e+05 1.09652418e+05 1.58478591e+05
 4.89950622e+03 1.66226506e+05 1.27341835e+05 1.35224816e+05
 5.55139555e+04 1.87389135e+05 9.95875183e+04 5.95028817e+04
 2.47505053e+04 1.50119739e+05 4.79174117e+03 1.14597830e+05
 4.94194297e+04 9.62943098e+04 4.52640946e+04 1.50871566e+05
 1.51722522e+05 8.53748279e+03 5.72649380e+04 8.62458044e+04
 6.98056889e+04 1.78918992e+05 5.28020315e+04 6.14224236e+04
 1.39378264e+05 1.61853726e+05 1.76831021e+05 1.46250041e+05
 1.82130540e+05 2.81466637e+03 1.77808492e+05 1.44659242e+05
 6.29198588e+03 1.81630960e+05 5.09973263e+02 2.31529086e+05
 8.53370781e+04 1.23877441e+05 1.46957355e+05 1.81660977e+05
 1.55226818e+05 2.024243

[7.57089260e+08 5.00134869e+08 2.44838962e+09 2.60761450e+09
 2.02426375e+09 5.10003655e+08 2.71757021e+09 1.03909005e+09
 2.94658775e+09 2.45808449e+09 4.52053708e+08 3.18429748e+09
 3.17105032e+09 3.69796520e+09 3.78546451e+09 5.05983993e+08
 3.64241899e+09 5.18379766e+08 1.88847820e+09 3.61892442e+09
 3.33244126e+09 4.78343574e+08 2.83023136e+09 4.50123393e+08
 1.87370816e+09 4.73896970e+08 2.82330232e+09 2.58374271e+09
 2.77829291e+09 4.12916349e+08 2.61284810e+09 2.81925136e+09
 7.24942957e+08 1.87069093e+09 2.17751496e+09 2.73481635e+09
 2.70603613e+09 2.76910330e+09 2.70591871e+09 1.85961351e+09
 2.22673908e+09 9.99514206e+08 2.75640174e+09 2.76478960e+09
 2.75787503e+09 3.97156982e+08 2.74099938e+09 2.71895230e+09
 2.41344527e+09 6.15972872e+08 4.34555281e+08 2.10733352e+09
 5.03632683e+08 2.69272423e+09 1.45570015e+09 9.45091201e+08
 1.63529834e+09 4.69644381e+08 7.63483568e+08 4.92247905e+08
 3.16248834e+09 2.49522118e+09 3.07316105e+09 5.27897461e+08
 2.05643714e+09 4.110386

[1.52184139e+11 2.76031543e+12 5.06907720e+11 5.43804576e+11
 5.94131460e+11 4.74677377e+12 6.57486329e+11 4.83851908e+12
 6.66183341e+11 6.63645583e+11 4.97996074e+12 6.79488160e+11
 6.75359290e+11 7.84398350e+11 8.01340135e+11 5.82636217e+12
 7.73030696e+11 6.02558418e+12 1.14624692e+12 7.66190503e+11
 7.07965933e+11 4.98353247e+12 6.04144323e+11 4.25451512e+12
 5.98692623e+11 4.29272287e+12 6.02840513e+11 5.83900278e+11
 5.91375586e+11 4.53397869e+12 6.26610493e+11 5.99125571e+11
 4.64545788e+12 6.16175862e+11 6.25683178e+11 5.79902372e+11
 5.76439787e+11 5.90001465e+11 5.76300610e+11 6.05309824e+11
 6.08073536e+11 4.82922300e+12 6.27520895e+11 5.87742587e+11
 5.87479788e+11 4.34529165e+12 5.83450223e+11 5.79018836e+11
 5.84509363e+11 4.51695966e+12 4.47306856e+12 6.22478498e+11
 4.71053964e+12 6.74939367e+11 2.76044383e+12 4.88499653e+12
 9.11487066e+11 4.91665167e+12 5.09827274e+12 5.36116558e+12
 6.96767250e+11 6.93356814e+11 6.64650361e+11 4.93422100e+12
 6.25676894e+11 4.765102

[7.97330579e+15 2.85706324e+15 2.77400109e+16 3.08508662e+16
 3.47676063e+16 5.14370832e+15 3.92682967e+16 5.28237145e+15
 4.01805792e+16 4.03109067e+16 5.46494815e+15 4.13832604e+16
 4.08675967e+16 4.70503443e+16 4.78181382e+16 6.34185943e+15
 4.62779851e+16 6.55623626e+15 4.63531934e+16 4.56993543e+16
 4.25304600e+16 5.46170290e+15 3.69350023e+16 4.67953048e+15
 3.66082184e+16 4.70690743e+15 3.65031413e+16 3.53262756e+16
 3.57314076e+16 4.95606016e+15 3.77374070e+16 3.60298450e+16
 5.06096503e+15 3.68245935e+16 3.76375000e+16 3.49739262e+16
 3.49133435e+16 3.57944307e+16 3.48655568e+16 3.64229687e+16
 3.64149373e+16 5.23643746e+15 3.74518915e+16 3.53774678e+16
 3.55459943e+16 4.75693440e+15 3.53113874e+16 3.50753816e+16
 3.54528362e+16 4.94428964e+15 4.89940418e+15 3.78089890e+16
 5.14635525e+15 4.04962705e+16 5.91442499e+15 5.32002252e+15
 4.05533483e+16 5.37978500e+15 5.56127705e+15 5.83510503e+15
 4.15985409e+16 4.17137175e+16 4.00132020e+16 5.39826979e+15
 3.80597256e+16 5.219745

...

[4.63589641e+23 1.65386749e+23 1.65625100e+24 1.88576209e+24
 2.17405454e+24 3.28228113e+23 2.53944295e+24 3.45517685e+23
 2.64687107e+24 2.67043189e+24 3.63533181e+23 2.74736311e+24
 2.70176701e+24 3.09434534e+24 3.12901924e+24 4.13501526e+23
 3.01722707e+24 4.26501485e+23 3.01907742e+24 2.98809819e+24
 2.80022543e+24 3.62651481e+23 2.46604697e+24 3.13539308e+23
 2.44922025e+24 3.13914921e+23 2.42810166e+24 2.34158576e+24
 2.36270626e+24 3.27042402e+23 2.48558416e+24 2.36903524e+24
 3.31919837e+23 2.41592745e+24 2.47400647e+24 2.30930647e+24
 2.31319426e+24 2.37370518e+24 2.30472926e+24 2.39474605e+24
 2.38383697e+24 3.41174598e+23 2.45042659e+24 2.32813771e+24
 2.35109085e+24 3.15270324e+23 2.34367199e+24 2.33013576e+24
 2.35574414e+24 3.27808706e+23 3.24791721e+23 2.49887408e+24
 3.39200602e+23 2.66322766e+24 3.69763559e+23 3.50384769e+23
 2.67643036e+24 3.54918013e+23 3.65311960e+23 3.81343930e+23
 2.72290452e+24 2.74050566e+24 2.63583831e+24 3.56792803e+23
 2.52133171e+24 3.461894

[1.59150480e+27 5.67183305e+26 5.72475098e+27 6.56004947e+27
 7.61560155e+27 1.15754527e+27 9.00770731e+27 1.23184858e+27
 9.46711346e+27 9.57408964e+27 1.30535658e+27 9.86013764e+27
 9.68643556e+27 1.10767769e+28 1.11814211e+28 1.47542775e+27
 1.07631771e+28 1.52011674e+27 1.07703297e+28 1.06775536e+28
 1.00303259e+28 1.30254859e+27 8.87413409e+27 1.12979392e+27
 8.82526601e+27 1.13038289e+27 8.73470614e+27 8.41101559e+27
 8.47765654e+27 1.17238449e+27 8.90406532e+27 8.48170629e+27
 1.18747834e+27 8.64546244e+27 8.85877697e+27 8.28023332e+27
 8.30217705e+27 8.52015896e+27 8.26417970e+27 8.57369862e+27
 8.52522346e+27 1.21862568e+27 8.76186188e+27 8.33873865e+27
 8.43504045e+27 1.13205909e+27 8.42085044e+27 8.37521566e+27
 8.46701568e+27 1.17708923e+27 1.16596054e+27 8.96174509e+27
 1.21569383e+27 9.53931863e+27 1.32396151e+27 1.25488252e+27
 9.58801789e+27 1.27096285e+27 1.30685085e+27 1.36270788e+27
 9.73565202e+27 9.81144869e+27 9.44367483e+27 1.27947074e+27
 9.04924122e+27 1.243010

[1.07688759e+30 3.83542712e+29 3.89016078e+30 4.47604816e+30
 5.22020566e+30 7.97092426e+29 6.22930346e+30 8.55131537e+29
 6.58848862e+30 6.67531282e+30 9.11235536e+29 6.88149487e+30
 6.75629217e+30 7.71818699e+30 7.78180558e+30 1.02576903e+30
 7.48153705e+30 1.05598747e+30 7.48733332e+30 7.43185093e+30
 6.99318140e+30 9.09857689e+29 6.20708341e+30 7.91022549e+29
 6.17953855e+30 7.91221455e+29 6.10975268e+30 5.87698194e+30
 5.91870279e+30 8.17925300e+29 6.20886776e+30 5.91202117e+30
 8.27317825e+29 6.02443990e+30 6.17553003e+30 5.77732591e+30
 5.79616537e+30 5.94843012e+30 5.76587725e+30 5.97600664e+30
 5.93831055e+30 8.48201980e+29 6.10255949e+30 5.81436345e+30
 5.88821754e+30 7.90722795e+29 5.88443875e+30 5.85399576e+30
 5.91790278e+30 8.22123336e+29 8.14197130e+29 6.25386551e+30
 8.48041369e+29 6.65163037e+30 9.22912941e+29 8.74775065e+29
 6.68441487e+30 8.85774274e+29 9.10213874e+29 9.48524868e+29
 6.77962864e+30 6.83901581e+30 6.58582362e+30 8.92780034e+29
 6.31813692e+30 8.681126

[1.15087022e+33 2.20033292e+34 4.16959806e+33 4.81122629e+33
 5.62961070e+33 4.70080984e+34 6.76258914e+33 5.11283638e+34
 7.18851827e+33 7.29440584e+33 5.50451276e+34 7.52684191e+33
 7.38738167e+33 8.43307533e+33 8.49521022e+33 6.16051620e+34
 8.16017416e+33 6.33385478e+34 8.16748914e+33 8.11453874e+33
 7.64532229e+33 5.50779419e+34 6.80252094e+33 4.80731535e+34
 6.77837511e+33 4.80784107e+34 6.69707987e+33 6.43658957e+33
 6.47812753e+33 4.94273831e+34 6.78926895e+33 6.46275002e+33
 4.98841407e+34 6.58431766e+33 6.75125516e+33 6.31986794e+33
 6.34312062e+33 6.50963521e+33 6.30690304e+33 6.53242192e+33
 6.48847279e+33 5.10761918e+34 6.66763402e+33 6.35786446e+33
 6.44399748e+33 4.78748964e+34 6.44479339e+33 6.41258928e+33
 6.48230855e+33 4.97636974e+34 4.92696947e+34 6.84221323e+33
 5.12379912e+34 7.27324680e+33 5.56900977e+34 5.27749616e+34
 7.30695619e+33 5.34067859e+34 5.48298269e+34 5.70897844e+34
 7.40558110e+33 7.47617336e+33 7.20189870e+33 5.39563199e+34
 6.91505647e+33 5.254278

/Users/sikai/Dropbox/Research/post_sum/code/regain/regain/covariance/equality_time_graphical_lasso.py:303: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")


[-1.7932898282047703e+303,
 -4.469903958048052e+305,
 -1.5604637299203803e+306,
 -3.288581313399307e+306,
 -4.6148609895520264e+306,
 -5.3985933633299074e+306,
 -5.931070701495488e+306,
 -6.625272261208059e+306,
 -6.489792399702904e+306,
 -6.738634346985379e+306,
 -6.52991066588284e+306,
 -6.754165522476111e+306,
 -7.047908678604985e+306,
 -5.851103000204279e+306,
 -5.848055418300424e+306,
 -5.642655317050465e+306,
 -5.695532160206416e+306,
 -5.3412548685082015e+306,
 -5.619300426063711e+306,
 -5.664032953742732e+306,
 -5.792097346700768e+306,
 -5.706488492836593e+306,
 -5.888066800983433e+306,
 -6.27036465943021e+306,
 -6.059973704508097e+306,
 -6.045984217887504e+306,
 -6.066704153457694e+306,
 -6.000577878474755e+306,
 -6.201288707415139e+306,
 -6.036055567484171e+306,
 -5.5748603247941146e+306,
 -6.139346427395343e+306,
 -6.148646656307753e+306,
 -6.212702097353928e+306,
 -6.022473488979261e+306,
 -6.56640871244614e+306,
 -6.758260279685672e+306,
 -6.172927935296258e+306,
 -6.53617

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
fig.patch.set_facecolor('white')
# colors = "bgrcmykw"
# color_index = 0
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.tab20.colors)

ax.plot(range(n_time), emp_inv_score, label=r'Empirical Inverse')
ax.plot(range(n_time), baseline_score, label=r'Constraint')
ax.plot(range(n_time), fit_score, label=r'TGL')

fig.legend(fontsize=15)
ax.set_ylabel('DTrace Loss', fontsize=15)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_xlabel('Time t', fontsize=15)
ax.set_title(r'DTrace Loss for Empirical Inverse, Constraint, TGL', fontsize=20)
plt.tight_layout()
# plt.savefig('diff_like_{}_{}.pdf'.format(dim_X, dim_fac))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
fig.patch.set_facecolor('white')
# colors = "bgrcmykw"
# color_index = 0
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.tab20.colors)

ax.plot(range(n_time), emp_inv_score, label=r'Empirical Inverse')
ax.plot(range(n_time), baseline_score, label=r'Constraint')
ax.plot(range(n_time), fit_score, label=r'TGL')

fig.legend(fontsize=15)
ax.set_ylabel('DTrace Loss', fontsize=15)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_xlabel('Time t', fontsize=15)
ax.set_title(r'DTrace Loss for Empirical Inverse, Constraint, TGL', fontsize=20)
plt.tight_layout()
# plt.savefig('diff_like_{}_{}.pdf'.format(dim_X, dim_fac))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
fig.patch.set_facecolor('white')
# colors = "bgrcmykw"
# color_index = 0

ax.plot(range(n_time), [sum(sum(abs(tgl.emp_inv[k]) > 0)) for k in range(n_time)], label=r'Empirical Inverse')
ax.plot(range(n_time), [sum(sum(abs(pre[k]) > 0)) for k in range(n_time)], label=r'TGL')

fig.legend(fontsize=15)
ax.set_ylabel('Sparsity', fontsize=15)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_xlabel('Time t', fontsize=15)
ax.set_title(r'Sparsity for Empirical Inverse and TGL', fontsize=20)
plt.tight_layout()
# plt.savefig('diff_supp_{}_{}.pdf'.format(dim_X, dim_fac))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
fig.patch.set_facecolor('white')
# colors = "bgrcmykw"
# color_index = 0
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.tab20.colors)

diff_fit = [norm(pre[k] - pre[k-1], 'fro') for k in range(1, 100)]
diff_emp = [norm(tgl.emp_inv[k] - tgl.emp_inv[k-1], 'fro') for k in range(1, 100)]
ax.plot(range(1, n_time), np.array(diff_fit), alpha=1, label=r'Empirical Inverse')
ax.plot(range(1, n_time), np.array(diff_emp), alpha=1, label=r'TGL')

        
fig.legend(fontsize=15)
ax.set_ylabel('Difference in Frobenius Norm', fontsize=15)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_xlabel('Time t', fontsize=15)
ax.set_title(r'Difference in Frobenius Norm for Empirical Inverse and TGL', fontsize=20)
plt.tight_layout()
# plt.savefig('diff_fro_{}_{}.pdf'.format(dim_X, dim_fac))